In [3]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

In [4]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import logging as lg
lg.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=lg.INFO)
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
import wiki
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

# General column meanings
* y_org: what the orginal time series data is (should have missing values staying as missing!)
* y: what the model was trained on (the y in val section is kinda irrelevant then)
* yhat_org: original unchanged output of the models predict function
* yhat: final predictions including post processing
# Scoring
* Should all be scored using y_org vs yhat
* SHOULD MAKE SURE y_org HASN't HAD fillna(0)

# AIM: Get new smape -> v2.6
* series from pagedf into y_org

## Version 4.1
Should set version directory name in next cell. Should describe version specifics (outliers, holidays, validation period)

* TRAINING
   * Val indexing on -60
   * Cut outliers out on upper 95% quartile `forecast.loc[forecast['yhat'] < 0,['yhat']] = 0.0`
   * Linear growth
   * Fill ALL other NaNs to 0 BUT KEEP y_org (get it from pagedf)
   * Now with try:except: for the `RuntimeError': k initialized to invalid value (-nan)` which replaces first `y` with 0.001
   * Set n changepoints evenly spaced change points -> `ds.ds[::n]`
   
* PREDICTIONS
   * Truncating predictions at 0 
   * Rounding to nearest int

### Remarks
* ?


In [5]:
PROPHET_PATH = '../data/prophet/'
RESULTS_PATH = 'results/'

In [6]:
TARGET_VERSION = 'v4.1/'
BASE_VERSION = 'v4/'
os.makedirs(PROPHET_PATH+TARGET_VERSION)

In [7]:
pagedf = pd.read_feather(PROPHET_PATH+'pagedf.f')

In [8]:
forecast_files = [x.split('/')[-1] for x in glob.glob(PROPHET_PATH+BASE_VERSION+'*df.f')]

In [9]:
forecast_files[18284]

'20274df.f'

In [19]:
pagedf[forecast_files[18284][:-4]].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: 20274, dtype: float64

In [11]:
[x[:-4] for x in forecast_files[:2]]

['61780', '88262']

In [12]:
base_df = pd.read_feather(PROPHET_PATH+BASE_VERSION+forecast_files[18284])

In [18]:
base_df.head()

,ds,t,trend,seasonal_lower,seasonal_upper,trend_lower,trend_upper,yhat_lower,yhat_upper,weekly,...,weekly_upper,yearly,yearly_lower,yearly_upper,seasonal,yhat,yhat_org,y,y_org,train
0,2015-07-01,0.000000,4.717449,-1.708587,-1.708587,4.717449,4.717449,0.251579,5.687724,-0.277639,...,-0.277639,-1.430948,-1.430948,-1.430948,-1.708587,3,3.008862,3.0,3.0,1.0
1,2015-07-02,0.002045,4.717523,-1.675377,-1.675377,4.717523,4.717523,0.339266,5.860680,-0.328690,...,-0.328690,-1.346687,-1.346687,-1.346687,-1.675377,3,3.042146,2.0,2.0,1.0
2,2015-07-03,0.004090,4.717596,-1.107914,-1.107914,4.717596,4.717596,0.816130,6.144053,0.147018,...,0.147018,-1.254931,-1.254931,-1.254931,-1.107914,4,3.609683,5.0,5.0,1.0
3,2015-07-04,0.006135,4.717670,-0.773410,-0.773410,4.717670,4.717670,1.041049,6.637009,0.383704,...,0.383704,-1.157115,-1.157115,-1.157115,-0.773410,4,3.944260,5.0,5.0,1.0
4,2015-07-05,0.008180,4.717744,-0.702107,-0.702107,4.717744,4.717744,1.280959,6.809764,0.352651,...,0.352651,-1.054758,-1.054758,-1.054758,-0.702107,4,4.015637,3.0,3.0,1.0


In [15]:
val_results = []
for file in tqdm(forecast_files):
    base_df = pd.read_feather(PROPHET_PATH+BASE_VERSION+file)
    # y_org to be the orginal timeseries data
    base_df.loc[:,'y_org'] = pagedf[file[:-4]]
    base_df.to_feather(PROPHET_PATH+TARGET_VERSION+file)
    full_smape = wiki.val.smape(base_df.y_org, base_df.yhat)
    val_smape = wiki.val.smape(base_df[base_df['train'] == 0].y_org,base_df[base_df['train'] == 0].yhat)
    val_results.append((file[:-4], full_smape, val_smape))
val_results = pd.DataFrame(val_results, columns=['page_index',TARGET_VERSION[:-1]+'_full',TARGET_VERSION[:-1]+'_val'])
val_results.to_feather(PROPHET_PATH+RESULTS_PATH+TARGET_VERSION[:-1]+'df.f')

100%|██████████| 145063/145063 [29:33<00:00, 81.79it/s]


In [16]:
resultsdf = pd.read_feather(PROPHET_PATH+RESULTS_PATH+TARGET_VERSION[:-1]+'df.f')

In [17]:
resultsdf['v4.1_val'].mean()
# base was v4 at 55.696

57.112413202609467